# Основний приклад AutoGen

У цьому прикладі коду ви використаєте [AutoGen](https://aka.ms/ai-agents/autogen) AI Framework для створення базового агента.

Мета цього прикладу — показати вам кроки, які ми пізніше використаємо в додаткових прикладах коду при реалізації різних агентних шаблонів.


## Імпорт необхідних пакетів Python


In [ ]:
import os
from dotenv import load_dotenv

from autogen_agentchat.agents import AssistantAgent
from autogen_core.models import UserMessage
from autogen_ext.models.azure import AzureAIChatCompletionClient
from azure.core.credentials import AzureKeyCredential
from autogen_core import CancellationToken

from autogen_agentchat.messages import TextMessage
from autogen_agentchat.ui import Console


## Створення клієнта

У цьому прикладі ми будемо використовувати [GitHub Models](https://aka.ms/ai-agents-beginners/github-models) для доступу до LLM.

`model` визначено як `gpt-4o-mini`. Спробуйте змінити модель на іншу, доступну на ринку GitHub Models, щоб побачити різні результати.

Для швидкого тесту ми просто запустимо простий запит - `What is the capital of France`.


In [ ]:
load_dotenv()
client = AzureAIChatCompletionClient(
    model="gpt-4o-mini",
    endpoint="https://models.inference.ai.azure.com",
    # To authenticate with the model you will need to generate a personal access token (PAT) in your GitHub settings.
    # Create your PAT token by following instructions here: https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/managing-your-personal-access-tokens
    credential=AzureKeyCredential(os.getenv("GITHUB_TOKEN")),
    model_info={
        "json_output": True,
        "function_calling": True,
        "vision": True,
        "family": "unknown",
    },
)

result = await client.create([UserMessage(content="What is the capital of France?", source="user")])
print(result)

## Визначення агента

Тепер, коли ми налаштували `client` і переконалися, що він працює, давайте створимо `AssistantAgent`. Кожному агенту можна призначити:  
**name** - Коротке ім'я, яке буде корисним для посилання на нього в багатозадачних потоках.  
**model_client** - Клієнт, якого ви створили на попередньому етапі.  
**tools** - Доступні інструменти, які агент може використовувати для виконання завдання.  
**system_message** - Метапідказка, яка визначає завдання, поведінку та тон LLM.  

Ви можете змінити системне повідомлення, щоб побачити, як LLM реагує. Ми розглянемо `tools` у Уроці №4.


In [ ]:
agent = AssistantAgent(
    name="assistant",
    model_client=client,
    tools=[],
    system_message="You are a travel agent that plans great vacations",
)

## Запустіть Агента

Наведена нижче функція запустить агента. Ми використовуємо метод `on_message`, щоб оновити стан Агента новим повідомленням.

У цьому випадку ми оновлюємо стан новим повідомленням від користувача, яке звучить як `"Сплануй мені чудову сонячну відпустку"`.

Ви можете змінити зміст повідомлення, щоб побачити, як LLM реагує по-іншому.


In [ ]:
from IPython.display import display, HTML


async def assistant_run():
    # Define the query
    user_query = "Plan me a great sunny vacation"

    # Start building HTML output
    html_output = "<div style='margin-bottom:10px'>"
    html_output += "<div style='font-weight:bold'>User:</div>"
    html_output += f"<div style='margin-left:20px'>{user_query}</div>"
    html_output += "</div>"

    # Execute the agent response
    response = await agent.on_messages(
        [TextMessage(content=user_query, source="user")],
        cancellation_token=CancellationToken(),
    )

    # Add agent response to HTML
    html_output += "<div style='margin-bottom:20px'>"
    html_output += "<div style='font-weight:bold'>Assistant:</div>"
    html_output += f"<div style='margin-left:20px; white-space:pre-wrap'>{response.chat_message.content}</div>"
    html_output += "</div>"

    # Display formatted HTML
    display(HTML(html_output))

# Run the function
await assistant_run()


---

**Відмова від відповідальності**:  
Цей документ був перекладений за допомогою сервісу автоматичного перекладу [Co-op Translator](https://github.com/Azure/co-op-translator). Хоча ми прагнемо до точності, будь ласка, майте на увазі, що автоматичні переклади можуть містити помилки або неточності. Оригінальний документ на його рідній мові слід вважати авторитетним джерелом. Для критичної інформації рекомендується професійний людський переклад. Ми не несемо відповідальності за будь-які непорозуміння або неправильні тлумачення, що виникають внаслідок використання цього перекладу.
